# IMPORTING MODULES 

In [1]:
try:
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json 
    from elasticsearch import helpers 
    import warnings
    warnings.filterwarnings("ignore")
    print("Loaded.....")    
except Exception as e:
    
    print("Some modules are missing{}".format(e))

Loaded.....


# READING FILE

In [2]:
df=pd.read_csv("movies.csv")
df1=df.to_dict("records")

In [3]:
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [4]:
ENDPOINT= "http://localhost:9200/"
es=Elasticsearch(timeout=600,hosts=ENDPOINT)

# CONVERTING TO ELK FORMAT

In [5]:
def generator(df1):
    for c,line in enumerate(df1):
        yield{
         '_index':'movies',
         '_type' :'_doc',
         '_id':line.get('movieId'),
         '_source':{
             'title':line.get('title',["No Data"]),
             'genres':line.get('genres',["No Data"])
                    }
             } 
      
gen=generator(df1)


In [6]:
next(gen)

{'_index': 'movies',
 '_type': '_doc',
 '_id': 1,
 '_source': {'title': 'Toy Story (1995)',
  'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}}

# Settings and Mappings



In [7]:
settings = {
    'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 1,
            "similarity": {
             "custom_bm25": {
                "type": "BM25",
                "b": 0,
                "k1" : 0.9
             }
          }
                
            }
    ,
    
    'mappings': {
        
            'properties': {
                'title': {
                    'type': 'text',
                    'analyzer': 'english'
                },
                "genres":{
                    "type": 'text',
                    'analyzer':'english'
                }
            }
        }
    }

In [8]:
my=es.indices.create(index="movies",ignore=[400,404],body=settings)

In [9]:
my

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [movies/-T4beOfJSz2XamYDmXaMLA] already exists',
    'index_uuid': '-T4beOfJSz2XamYDmXaMLA',
    'index': 'movies'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [movies/-T4beOfJSz2XamYDmXaMLA] already exists',
  'index_uuid': '-T4beOfJSz2XamYDmXaMLA',
  'index': 'movies'},
 'status': 400}

# UPLOAD TO ES 

In [13]:
try:
    res=helpers.bulk(es,generator(df1))
    print("Working")
except Exception as e:
    print("Done")

Working


# USER QUERY

In [12]:
user_request=input("What are you looking for?")
query_body = {
  "query": {
      "multi_match":{
          "query":user_request,
          "fields":["title" ,"genres"]
      }
  }
}


# Pass the query dictionary to the 'body' parameter of the
# client's Search() method, and have it return results:
result = es.search(index="movies", body=query_body,size=10000,)
print ("total hits:", len(result["hits"]["hits"]),"\n")

#result     for full info about the results

for i in result['hits']['hits']:
    print(i['_source']['title']+" "+"\n"+i['_source']['genres']+"\n")

What are you looking for?batman
total hits: 12 

Batman (1989) 
Action|Crime|Thriller

Batman (1966) 
Action|Adventure|Comedy

Batman Forever (1995) 
Action|Adventure|Comedy|Crime

Batman Returns (1992) 
Action|Crime

Batman & Robin (1997) 
Action|Adventure|Fantasy|Thriller

Batman Begins (2005) 
Action|Crime|IMAX

Batman: Mask of the Phantasm (1993) 
Animation|Children

Batman: Year One (2011) 
Action|Animation|Crime

Batman: Under the Red Hood (2010) 
Action|Animation

Batman v Superman: Dawn of Justice (2016) 
Action|Adventure|Fantasy|Sci-Fi

Batman: The Dark Knight Returns, Part 1 (2012) 
Action|Animation|Sci-Fi

Batman: The Dark Knight Returns, Part 2 (2013) 
Action|Animation

